# Типичные задачи Excel, продемонстрированные в pandas

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D0%BA%D0%B5%D0%B9%D1%81%D1%8B%20%D0%BF%D0%BE%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D1%83%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/%D0%B7%D0%B0%D1%80%D0%BF%D0%BB%D0%B0%D1%82%D1%8B%20%D0%B2%20%D0%98%D0%A2.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

## Введение

Цель этой статьи - показать ряд повседневных задач `Excel` и то, как они выполняются в `pandas`. Некоторые примеры тривиальны, но я думаю, важно представить как простые, так и более сложные функции. 

В качестве дополнительного бонуса я собираюсь выполнить нечеткое сопоставление строк (`fuzzy string matching`), чтобы продемонстрировать, как `pandas` могут использовать модули `Python`. 

> оригинал статьи Криса [тут](https://pbpython.com/excel-pandas-comp.html)

Разберемся? Давайте начнем.

## Добавление суммы в строку 

Первая задача, которую я покажу, - это суммирование нескольких столбцов для добавления итогового столбца.

Начнем с импорта данных из `Excel` в кадр данных `pandas`:

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel("https://github.com/dm-fedorov/pandas_basic/blob/master/%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%B5%20%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20pandas/data/excel-comp-data.xlsx?raw=True")
df.head()

,account,name,street,city,state,postal-code,Jan,Feb,Mar
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752,10000,62000,35000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365,95000,45000,35000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517,91000,120000,35000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021,45000,120000,10000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681,162000,120000,35000


Мы хотим добавить столбец с итогами, чтобы показать общие продажи за январь, февраль и март. Это просто сделать в `Excel` и в `pandas`. 

Для `Excel` я добавил формулу `SUM(G2:I2)` в столбец `J`. 

Вот как это выглядит:

![](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/excel-1.png)      

Далее, вот как это делается в `pandas`:

In [2]:
df["total"] = df["Jan"] + df["Feb"] + df["Mar"]
df.head()

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752,10000,62000,35000,107000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365,95000,45000,35000,175000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517,91000,120000,35000,246000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021,45000,120000,10000,175000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681,162000,120000,35000,317000


Затем получим итоговые и некоторые другие значения за каждый месяц. 

Пытаемся сделать в `Excel`:

![](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/excel-2.png)

Как видите, мы добавили `SUM(G2:G16)` в строку `17` в каждом столбце, чтобы получить итоги по месяцам. 

В `pandas` легко выполнять анализ на уровне столбцов. Вот пара примеров:

In [3]:
df["Jan"].sum(), df["Jan"].mean(), df["Jan"].min(), df["Jan"].max()

(1462000, 97466.66666666667, 10000, 162000)

Теперь хотим в `pandas` сложить сумму по месяцам с итогом (`total`). 

Здесь `pandas` и `Excel` немного расходятся. В `Excel` очень просто складывать итоги в ячейках за каждый месяц. 

Поскольку `pandas` необходимо поддерживать целостность всего [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), то придется добавить еще пару шагов.

Сначала создайте сумму для столбцов по месяцам и итога (`total`).

In [4]:
sum_row = df[["Jan", "Feb", "Mar", "total"]].sum()
sum_row

Jan      1462000
Feb      1507000
Mar       717000
total    3686000
dtype: int64

Интуитивно понятно, если вы хотите добавить итоги в виде строки, то нужно проделать некоторые незначительные манипуляции.

Для начала - транспонировать данные и преобразовать [`Series`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) в [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), чтобы было проще объединить существующие данные. 

Атрибут [`T`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.T.html) позволяет преобразовать данные из строк в столбцы.

In [5]:
df_sum = pd.DataFrame(data=sum_row).T
df_sum

,Jan,Feb,Mar,total
0,1462000,1507000,717000,3686000


Последнее, что нужно сделать перед суммированием итогов, - это добавить недостающие столбцы. 

Для этого используем функцию [`reindex`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html). 

Хитрость заключается в том, чтобы добавить все наши столбцы, а затем разрешить `pandas` заполнить отсутствующие значения.

In [6]:
df_sum = df_sum.reindex(columns=df.columns)
df_sum

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
0,NaN,NaN,NaN,NaN,NaN,NaN,1462000,1507000,717000,3686000


Теперь, когда у нас есть красиво отформатированный `DataFrame`, можем добавить его к существующему, используя метод [`append`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html):

In [7]:
df_final = df.append(df_sum, ignore_index=True)
df_final.tail(3)

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
13,268755.0,Walsh-Haley,2624 Beatty Parkways,Goodwinmouth,RhodeIsland,31919.0,55000,120000,35000,210000
14,273274.0,McDermott PLC,8917 Bergstrom Meadow,Kathryneborough,Delaware,27933.0,150000,120000,70000,340000
15,NaN,NaN,NaN,NaN,NaN,NaN,1462000,1507000,717000,3686000


## Дополнительные преобразования данных

В качестве примера попробуем добавить к набору данных аббревиатуру штата.

С точки зрения `Excel`, самый простой способ - это добавить новый столбец, выполнить `vlookup` ([ВПР](https://support.microsoft.com/ru-ru/office/%D0%B2%D0%BF%D1%80-%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F-%D0%B2%D0%BF%D1%80-0bbc8083-26fe-4963-8ab8-93a18ad188a1)) по имени штата и заполнить аббревиатуру.

Вот снимок того, как выглядят результаты:

![](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/excel-3.png)

Вы заметите, что после выполнения `vlookup` ряд значений отображаются неправильно. Это потому, что мы неправильно написали некоторые штаты. Обработать это в `Excel` для больших наборов данных сложно.

В `pandas` у нас есть вся мощь экосистемы `Python`. Размышляя о том, как решить эту проблему с грязными данными, я подумал о попытке сопоставления нечеткого текста (`fuzzy text matching`), чтобы определить правильное значение.

К счастью, кто-то проделал большую работу в этом направлении. 

В библиотеке [`fuzzy wuzzy`](https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/) есть несколько довольно полезных функций для таких ситуаций.

> fuzzywuzzy использует [расстояние Левенштейна](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D1%88%D1%82%D0%B5%D0%B9%D0%BD%D0%B0) для вычисления различий между последовательностями.

> см. [Применение библиотеки FuzzyWuzzy для нечёткого сравнения в Python](https://habr.com/ru/post/491448/) на Хабре

In [8]:
#!pip3 install fuzzywuzzy

In [9]:
#!pip install python-Levenshtein

Начнем с импорта соответствующих нечетких функций:

In [10]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Другой фрагмент кода, который нам нужен, - это отображение имени штата в аббревиатуру. Вместо того, чтобы пытаться напечатать его самостоятельно, небольшой поиск в Google подсказал следующий код:

In [11]:
state_to_code = {"VERMONT": "VT", "GEORGIA": "GA", "IOWA": "IA", "Armed Forces Pacific": "AP", "GUAM": "GU",
                 "KANSAS": "KS", "FLORIDA": "FL", "AMERICAN SAMOA": "AS", "NORTH CAROLINA": "NC", "HAWAII": "HI",
                 "NEW YORK": "NY", "CALIFORNIA": "CA", "ALABAMA": "AL", "IDAHO": "ID", "FEDERATED STATES OF MICRONESIA": "FM",
                 "Armed Forces Americas": "AA", "DELAWARE": "DE", "ALASKA": "AK", "ILLINOIS": "IL",
                 "Armed Forces Africa": "AE", "SOUTH DAKOTA": "SD", "CONNECTICUT": "CT", "MONTANA": "MT", "MASSACHUSETTS": "MA",
                 "PUERTO RICO": "PR", "Armed Forces Canada": "AE", "NEW HAMPSHIRE": "NH", "MARYLAND": "MD", "NEW MEXICO": "NM",
                 "MISSISSIPPI": "MS", "TENNESSEE": "TN", "PALAU": "PW", "COLORADO": "CO", "Armed Forces Middle East": "AE",
                 "NEW JERSEY": "NJ", "UTAH": "UT", "MICHIGAN": "MI", "WEST VIRGINIA": "WV", "WASHINGTON": "WA",
                 "MINNESOTA": "MN", "OREGON": "OR", "VIRGINIA": "VA", "VIRGIN ISLANDS": "VI", "MARSHALL ISLANDS": "MH",
                 "WYOMING": "WY", "OHIO": "OH", "SOUTH CAROLINA": "SC", "INDIANA": "IN", "NEVADA": "NV", "LOUISIANA": "LA",
                 "NORTHERN MARIANA ISLANDS": "MP", "NEBRASKA": "NE", "ARIZONA": "AZ", "WISCONSIN": "WI", "NORTH DAKOTA": "ND",
                 "Armed Forces Europe": "AE", "PENNSYLVANIA": "PA", "OKLAHOMA": "OK", "KENTUCKY": "KY", "RHODE ISLAND": "RI",
                 "DISTRICT OF COLUMBIA": "DC", "ARKANSAS": "AR", "MISSOURI": "MO", "TEXAS": "TX", "MAINE": "ME"}

Вот несколько примеров того, как работает функция сопоставления нечеткого текста:

In [12]:
process.extractOne("Minnesotta", choices=state_to_code.keys())

# ('результат', индекс сходства)

('MINNESOTA', 95)

In [13]:
process.extractOne("AlaBAMMazzz", choices=state_to_code.keys(), score_cutoff=80)

Теперь, когда мы знаем, как это работает, создаем функцию, которая берет столбец штата и преобразует его в допустимое сокращение. 

Для этих данных используем *порог наилучшего результата совпадения* `score_cutoff=80`. Можете поиграть с этим значением, чтобы увидеть, какое число подходит для ваших данных. 

В функции мы либо возвращаем допустимое сокращение, либо `np.nan`, чтобы у нас были допустимые значения в поле.

In [14]:
def convert_state(row):
    if pd.notnull(row['state']):
        abbrev = process.extractOne(row["state"], choices=state_to_code.keys(), score_cutoff=80)
        if abbrev:
            return state_to_code[abbrev[0]]
    return np.nan

Добавьте столбец в нужном месте и заполните его значениями `NaN`:

In [15]:
df_final.insert(6, "abbrev", np.nan)
df_final.head()

,account,name,street,city,state,postal-code,abbrev,Jan,Feb,Mar,total
0,211829.0,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752.0,NaN,10000,62000,35000,107000
1,320563.0,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365.0,NaN,95000,45000,35000,175000
2,648336.0,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517.0,NaN,91000,120000,35000,246000
3,109996.0,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021.0,NaN,45000,120000,10000,175000
4,121213.0,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681.0,NaN,162000,120000,35000,317000


Теперь используем [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) для добавления сокращений в столбец `abbrev`:

In [16]:
df_final['abbrev'] = df_final.apply(convert_state, axis=1)
df_final.tail()

,account,name,street,city,state,postal-code,abbrev,Jan,Feb,Mar,total
11,231907.0,Hahn-Moore,18115 Olivine Throughway,Norbertomouth,NorthDakota,31415.0,ND,150000,10000,162000,322000
12,242368.0,"Frami, Anderson and Donnelly",182 Bertie Road,East Davian,Iowa,72686.0,IA,162000,120000,35000,317000
13,268755.0,Walsh-Haley,2624 Beatty Parkways,Goodwinmouth,RhodeIsland,31919.0,RI,55000,120000,35000,210000
14,273274.0,McDermott PLC,8917 Bergstrom Meadow,Kathryneborough,Delaware,27933.0,DE,150000,120000,70000,340000
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1462000,1507000,717000,3686000


Думаю, это круто! 

Мы разработали очень простой процесс для очистки данных. Очевидно, когда у вас 15 строк, это не имеет большого значения. Однако что, если бы у вас было 15 000? 

## Промежуточные итоги

В последнем разделе этой статьи давайте рассмотрим промежуточные итоги (`subtotal`) по штатам.

В `Excel` мы бы использовали инструмент `subtotal`:

![](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/excel-4.png)

Результат будет выглядеть так:

![](https://pbpython.com/images/excel-5.png)

Создание промежуточного итога в `pandas` выполняется с помощью метода [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html):

In [17]:
df_sub = df_final[["abbrev", "Jan", "Feb", "Mar", "total"]].groupby('abbrev').sum()
df_sub

,Jan,Feb,Mar,total
abbrev,,,,
AR,150000,120000,35000,305000
CA,162000,120000,35000,317000
DE,150000,120000,70000,340000
IA,253000,240000,70000,563000
ID,70000,120000,35000,225000
ME,45000,120000,10000,175000
MS,62000,120000,70000,252000
NC,95000,45000,35000,175000
ND,150000,10000,162000,322000


Затем хотим отобразить данные с обозначением валюты, используя [`applymap`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.applymap.html) для всех значений в кадре данных:

In [18]:
def money(x):
    return "${:,.0f}".format(x)

formatted_df = df_sub.applymap(money)
formatted_df

,Jan,Feb,Mar,total
abbrev,,,,
AR,"$150,000","$120,000","$35,000","$305,000"
CA,"$162,000","$120,000","$35,000","$317,000"
DE,"$150,000","$120,000","$70,000","$340,000"
IA,"$253,000","$240,000","$70,000","$563,000"
ID,"$70,000","$120,000","$35,000","$225,000"
ME,"$45,000","$120,000","$10,000","$175,000"
MS,"$62,000","$120,000","$70,000","$252,000"
NC,"$95,000","$45,000","$35,000","$175,000"
ND,"$150,000","$10,000","$162,000","$322,000"


Форматирование выглядит неплохо, теперь можем получить итоговые значения, как раньше:

In [19]:
sum_row = df_sub[["Jan", "Feb", "Mar", "total"]].sum()
sum_row

Jan      1462000
Feb      1507000
Mar       717000
total    3686000
dtype: int64

Преобразуйте значения в столбцы и отформатируйте их:

In [20]:
df_sub_sum = pd.DataFrame(data=sum_row).T
df_sub_sum = df_sub_sum.applymap(money)
df_sub_sum

,Jan,Feb,Mar,total
0,"$1,462,000","$1,507,000","$717,000","$3,686,000"


Наконец, добавьте итоговое значение в `DataFrame`:

In [21]:
final_table = formatted_df.append(df_sub_sum)
final_table

,Jan,Feb,Mar,total
AR,"$150,000","$120,000","$35,000","$305,000"
CA,"$162,000","$120,000","$35,000","$317,000"
DE,"$150,000","$120,000","$70,000","$340,000"
IA,"$253,000","$240,000","$70,000","$563,000"
ID,"$70,000","$120,000","$35,000","$225,000"
ME,"$45,000","$120,000","$10,000","$175,000"
MS,"$62,000","$120,000","$70,000","$252,000"
NC,"$95,000","$45,000","$35,000","$175,000"
ND,"$150,000","$10,000","$162,000","$322,000"
PA,"$70,000","$95,000","$35,000","$200,000"


Вы заметите, что для итоговой строки индекс равен `0`. 

Можем изменить это с помощью метода [`rename`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html):

In [22]:
final_table = final_table.rename(index={0:"Total"})
final_table

,Jan,Feb,Mar,total
AR,"$150,000","$120,000","$35,000","$305,000"
CA,"$162,000","$120,000","$35,000","$317,000"
DE,"$150,000","$120,000","$70,000","$340,000"
IA,"$253,000","$240,000","$70,000","$563,000"
ID,"$70,000","$120,000","$35,000","$225,000"
ME,"$45,000","$120,000","$10,000","$175,000"
MS,"$62,000","$120,000","$70,000","$252,000"
NC,"$95,000","$45,000","$35,000","$175,000"
ND,"$150,000","$10,000","$162,000","$322,000"
PA,"$70,000","$95,000","$35,000","$200,000"


> Модуль [`sidetable`](https://dfedorov.spb.ru/pandas/%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%20%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%8B%D1%85%20%D1%81%D0%B2%D0%BE%D0%B4%D0%BD%D1%8B%D1%85%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%20%D0%B2%20pandas%20%D1%81%20%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E%20sidetable.html) значительно упрощает этот процесс и делает его более надежным.

## Заключение

К настоящему времени большинство людей знают, что `pandas` умеет выполнять множество сложных манипуляций с данными подобно `Excel`. Изучая `pandas`, я все еще пытаюсь вспомнить, как это сделать в `Excel`. Понимаю, что это сравнение может быть не совсем справедливым - это разные инструменты. Однако я надеюсь достучаться до людей, которые знают `Excel` и хотят узнать, какие существуют альтернативы для их потребностей в обработке данных. Надеюсь, эти примеры помогут почувствовать уверенность в том, что можно заменить множество бесполезных манипуляций с данными в `Excel` с помощью pandas.

> В качестве бонуса рекомендую видео [Excel is Evil - Why it has no place in research](https://youtu.be/-NuTlczV72Q)